<a href="https://colab.research.google.com/github/manhhung14/echocardiograms/blob/main/Project_Machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Khởi tạo thư viện**

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch
from torch import nn
from torch.utils.data import DataLoader


**Lấy dữ liệu**

In [ ]:
!git clone https://github.com/manhhung14/echocardiograms
traindir = "/content/echocardiograms/DATA_CHAMBER_2021/train"
testdir = "/content/echocardiograms/DATA_CHAMBER_2021/test"

Cloning into 'echocardiograms'...
remote: Enumerating objects: 8344, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 8344 (delta 1), reused 11 (delta 0), pack-reused 8328
Receiving objects: 100% (8344/8344), 488.03 MiB | 34.63 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (8328/8328), done.


**Chuẩn bị dữ liệu**

In [ ]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def raw_image(size):
  transform_train = transforms.Compose([
    transforms.Resize(size=(size,size)),
    transforms.ToTensor()
  ])
  transform_test = transforms.Compose([
    transforms.Resize(size=(size,size)), 
    transforms.ToTensor()
  ])
  return transform_train, transform_test

def preprocess_image(size):
  transform_train = transforms.Compose([
    transforms.Resize(size=(size,size)),
    transforms.RandomEqualize(p=0.5),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor()
    ])
  transform_test = transforms.Compose([
    transforms.Resize(size=(size,size)),
    transforms.RandomEqualize(p=0.5),
    transforms.GaussianBlur(kernel_size=3), 
    transforms.ToTensor()
  ])
  return transform_train, transform_test

def augmentation_image(size):
  transform_train = transforms.Compose([
    transforms.Resize(size=(size,size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor()
  ])
  transform_test = transforms.Compose([
    transforms.Resize(size=(size,size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=10), 
    transforms.ToTensor()
  ])
  return transform_train, transform_test

def prepare_data(size):
  #transform_train, transform_test = raw_image(size)
  # transform_train, transform_test = preprocess_image(size)
  transform_train, transform_test = augmentation_image(size)
  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=32, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=32, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)



**Train & Test**

In [ ]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  final_loss = 0.0
  reporting_steps = 40
  step = 0
  for images, labels in loader:
    step += 1
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    final_loss = loss.item()
    running_loss += final_loss
    if step % reporting_steps == reporting_steps - 1:
      print(f"Epoch {epoch} step {step} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  return final_loss

def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  with torch.no_grad():
    model.eval()
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue


**Thực thi**

In [ ]:
def main(model_in, size):
  PATH = "./" + model_in + '.pth'
  classes = get_classes()
  class_out = len(classes)

  #transforms_train, transforms_test = raw_image(size)
  #transforms_train, transforms_test = preprocess_image(size)
  transforms_train, transforms_test = augmentation_image(size)
  datasets = prepare_data(size=size)
  loaders = prepare_loader(datasets)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  print("image size: ", datasets.train[0][0].shape)
  print("device:", device)
  print("model:", model_in)

  if model_in == 'vgg16':  
    model = torchvision.models.vgg16(pretrained=False)
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=class_out, bias=True)
  elif model_in == 'resnet50':
    model = torchvision.models.resnet50(pretrained=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=class_out, bias=True) 
  elif model_in == 'resnet18':
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=512, out_features=class_out, bias=True) 
  elif model_in == 'densenet121': 
    model = torchvision.models.densenet121(pretrained=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=class_out, bias=True)
  else:
    pass
    
  model.to(device)
  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  accuracies = []
  losses = []
  for epoch in range(10):
    print("---------------------------------------------------------------")
    loss = train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    ypred_test, ytrue_test = test_epoch(epoch, model, loaders.test, device)

    print(classification_report(ytrue_test, ypred_test, target_names=classes))
    torch.save(model.state_dict(), PATH)

    # calculate report
    ypred_test = np.array(ypred_test)
    ytrue_test = np.array(ytrue_test)
    accuracy = int((ytrue_test==ypred_test).sum() / len(ytrue_test) *100)
    accuracies.append(accuracy)
    losses.append(round(loss, 4))

    
  print("model:", model_in, ", size:", size)
  print("accr: ", accuracies)
  print("loss: ", losses)
  return model

In [ ]:
model1 = main('resnet50', 224)
#model2 = main('resnet50', 64)
#model3 = main('resnet50', 32)

